In [46]:
import pandas as pd
import numpy as np
from datetime import datetime

In [66]:
df = pd.read_csv('raw_logs.csv',header=0).groupby('session')
for session, sdf in df:
    timeStr = datetime.fromtimestamp(session/1000.0).strftime('%Y-%m-%d %H:%M:%S')
    print('session {0} ({1}), {2} records'.format(session, timeStr, len(sdf.index)))
    
    # Sort by timestamp get the final average MPG for every trip (kff1205)
    sdf = sdf.sort_values('time')
    
    # Remove invalid values
    sdf = sdf[sdf.k43 != 0]

session 1509673999804 (2017-11-02 20:53:19), 1678 records


AttributeError: 'DataFrame' object has no attribute 'sort'

In [65]:
sdf.sort_values('time')

,v,session,id,time,kff1005,kff1006,kff1001,kff1007,k4,k2f,...,kff1221,k46,eml,k43,kff1271,k1f,kff1205,kff1204,kff1266,kff1270
2641,8,1510237809601,e74285c152b2a4640f2225494159cd65,1510238178557,0.0000,0.0000,0.000,0.0,0,0.0000,...,0,0,chenchen.bme@gmail.com,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
2640,8,1510237809601,e74285c152b2a4640f2225494159cd65,1510238178558,0.0000,0.0000,0.000,0.0,0,0.0000,...,0,0,chenchen.bme@gmail.com,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
2639,8,1510237809601,e74285c152b2a4640f2225494159cd65,1510238178559,0.0000,0.0000,0.000,0.0,0,0.0000,...,0,0,chenchen.bme@gmail.com,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0
2642,8,1510237809601,e74285c152b2a4640f2225494159cd65,1510238178559,-87.6241,41.8671,4.356,0.5,0,29.8039,...,0,0,chenchen.bme@gmail.com,14.901960,0.171592,395.0,13.885695,0.523091,348.0,0
2643,8,1510237809601,e74285c152b2a4640f2225494159cd65,1510238179521,-87.6241,41.8671,4.356,0.5,0,29.8039,...,0,0,chenchen.bme@gmail.com,14.901960,0.171783,396.0,13.884131,0.523091,349.0,0
2644,8,1510237809601,e74285c152b2a4640f2225494159cd65,1510238180521,-87.6241,41.8671,4.356,0.5,0,29.8039,...,0,0,chenchen.bme@gmail.com,14.901960,0.171916,397.0,13.882572,0.523091,350.0,0
2645,8,1510237809601,e74285c152b2a4640f2225494159cd65,1510238181521,-87.6241,41.8671,4.356,0.5,0,29.8039,...,0,0,chenchen.bme@gmail.com,14.901960,0.172113,398.0,13.879465,0.523091,351.0,0
2646,8,1510237809601,e74285c152b2a4640f2225494159cd65,1510238182522,-87.6241,41.8671,4.356,0.5,0,29.8039,...,0,0,chenchen.bme@gmail.com,14.901960,0.172251,399.0,13.877917,0.523091,352.0,0
2647,8,1510237809601,e74285c152b2a4640f2225494159cd65,1510238183521,-87.6241,41.8671,4.356,0.5,0,29.8039,...,0,0,chenchen.bme@gmail.com,14.901960,0.172501,400.0,13.876373,0.523091,353.0,0
2648,8,1510237809601,e74285c152b2a4640f2225494159cd65,1510238184521,-87.6241,41.8671,4.356,0.5,0,29.8039,...,0,0,chenchen.bme@gmail.com,14.901960,0.172721,401.0,13.874833,0.523091,354.0,0
